In [ ]:
import qubic.toolchain as tc
import qubic.rpc_client as rc
import qubitconfig.qchip as qc
from distproc.hwconfig import FPGAConfig, load_channel_configs
import numpy as np
import matplotlib.pyplot as plt

## Load Configs

Define FPGA config; this has timing information for the scheduler. For now it is fine to use the following default config. After that, load channel configs (firmware channel mapping + configuration, see [Understanding Channel Configuration](https://gitlab.com/LBL-QubiC/software/-/wikis/Understanding-Channel-Configuration) for details)

In [ ]:
fpga_config = FPGAConfig()
channel_configs = load_channel_configs('channel_config.json')

## Define CW pulse and Compile & Assemble

CW stands for continuous wave, and is a toggleable pulse. That is, once you turn on the CW pulse, it will generate a pulse until a new pulse is called. Calling a CW pulse is similar to a regular pulse, except the `'env'` parameter is `'cw'`.

In the following quantum circuit, we will call a CW pulse and let it run for 50 clock cycles before calling a non-CW pulse. This example if done at the pulse-level without referencing calibrated gates/parameters. Details on the QubiC circuit format and supported operations can be found [here](https://lbl-qubic.gitlab.io/distributed_processor/)

In [ ]:
circuit = [
    # Play the CW pulse
    {'name': 'pulse', 'phase': 0, 'freq': 6658138379, 'amp': 0.5, 'twidth': 0,
     'env': 'cw', 'dest': 'Q0.qdrv'},

    # Allow CW pulse to run for 50 clock cycles
    {'name': 'delay', 't': 100.e-9}, 

    # Play the second pulse
    {'name': 'pulse', 'phase': 0, 'freq': 6658138379, 'amp': 1, 'twidth': 24e-9,
     'env': {'env_func': 'square', 'paradict': {'phase': 0.0, 'amplitude': 1.0}},
     'dest': 'Q0.qdrv'}
]

Instead of using a delay, the twidth parameter can also be used to control the length of a CW segment. So the following circuit is equivalent to the circuit defined above:

In [ ]:
circuit = [
    # Play the CW pulse
    {'name': 'pulse', 'phase': 0, 'freq': 6658138379, 'amp': 0.5, 'twidth': 100.e-9, 
     'env': 'cw', 'dest': 'Q0.qdrv'}, # set the twidth parameter instead of using a delay

    # Play the second pulse
    {'name': 'pulse', 'phase': 0, 'freq': 6658138379, 'amp': 1, 'twidth': 24e-9,
     'env': {'env_func': 'square', 'paradict': {'phase': 0.0, 'amplitude': 1.0}},
     'dest': 'Q0.qdrv'}
]

In [ ]:
compiled_prog = tc.run_compile_stage(circuit, fpga_config, None, 
                                     compiler_flags={'resolve_gates': False})
compiled_prog.program
raw_asm = tc.run_assemble_stage(compiled_prog, channel_configs)

If no pulse is played on the same channel after the CW pulse, a CW signal will play indefinitely, and the program will not halt, resulting in a timeout error. 

In [ ]:
circuit = [
    # Play the CW pulse
    {'name': 'pulse', 'phase': 0, 'freq': 6658138379, 'amp': 0.5, 'twidth': 0,
     'env': 'cw', 'dest': 'Q0.qdrv'}]
# this will timeout, but the CW pulse will keep playing indefinitely, 
# which can be useful for debugging.

## Connect to Server and Run Circuit

Now that we've defined our circuit and compiled it to machine code, we can submit it to the ZCU216 and run it. Connect an osciliscope to the signal generating channel to visualize the pulses that are being generated.

In [ ]:
# Instantiate the runner client:
runner = rc.CircuitRunnerClient(ip='192.168.1.122', port=9095)

# Submit the circuit to the server, and run it once. 
s11 = runner.run_circuit_batch([raw_asm], 1)

You should be able to visualize the CW pulse on the osciliscope. Notice how it runs until the second pulse is called.

## Define DC pulse and Compile & Assemble

DC pulses need to be played on DC channels. Similar to a CW pulse, the DC pulse will play at the specified voltage level until another DC pulse is called. When calling a DC pulse, you are able to leave the frequency, envelope, and phase parameters as None, since they do not affect the DC pulse. The following circuit demonstrates the DC capability.

In [ ]:
prog = [{'name': 'pulse', 'dest': 'C0.dc', 'twidth': 0, 'amp': 0.5,
             'freq': None, 'phase': 0, 'env': None},
        
             {'name': 'delay', 't': 20.e-9, 'scope': ['C0.dc']},
        
             {'name': 'pulse', 'dest': 'C0.dc', 'twidth': 0, 'amp': 0.0,
              'freq': None, 'phase': 0, 'env': None},
        
             {'name': 'delay', 't': 20.e-9, 'scope': ['C0.dc']},
        
             {'name': 'pulse', 'dest': 'C0.dc', 'twidth': 0, 'amp': 1.0,
              'freq': None, 'phase': 0, 'env': None},
 ]

As with CW, DC pulses can replace the `delay` instruction with a nonzero `'twidth'` parameter.

In [ ]:
compiled_prog = tc.run_compile_stage(prog, fpga_config=fpga_config, qchip=None, compiler_flags={'resolve_gates': False},
                                    proc_grouping=[('{qubit}.qdrv', '{qubit}.rdrv', '{qubit}.rdlo'),
                                                    ('{qubit}.qdrv2',),
                                                    ('{qubit}.cdrv', '{qubit}.dc')]) # Resolve gates = false, dont give it the qchip
raw_asm = tc.run_assemble_stage(compiled_prog, channel_configs)

Note that the proc_grouping grouping parameter is required to introduce DC channels to the compiler. Now connect to the server and run the circuit with the next code block.

In [ ]:
# Instantiate the runner client:
runner = rc.CircuitRunnerClient(ip='192.168.1.122', port=9095)

# Submit the circuit to the server, and run it once. 
s11 = runner.run_circuit_batch([raw_asm], 1)